In [127]:
#Контрольная работа "SparkSQL"
#выполнялась в jn на образе jupiter/pyspark-notebook 

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import Window as w

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

df = (spark.read.option('header', True).option('sep',',').option('inferSchema', True).csv('owid-covid-data.csv'))

In [174]:
#Задание 1

df.select(df.iso_code, 'location', f.format_number(df.total_cases_per_million/10000.0, 2)
          .alias('percent')).where(df.date == '2021-03-31').sort(df.total_cases_per_million.desc()).show(15)

+--------+-------------+-------+
|iso_code|     location|percent|
+--------+-------------+-------+
|     AND|      Andorra|  15.54|
|     MNE|   Montenegro|  14.52|
|     CZE|      Czechia|  14.31|
|     SMR|   San Marino|  13.94|
|     SVN|     Slovenia|  10.37|
|     LUX|   Luxembourg|   9.85|
|     ISR|       Israel|   9.63|
|     USA|United States|   9.20|
|     SRB|       Serbia|   8.83|
|     BHR|      Bahrain|   8.49|
|     PAN|       Panama|   8.23|
|     PRT|     Portugal|   8.06|
|     EST|      Estonia|   8.02|
|     SWE|       Sweden|   7.97|
|     LTU|    Lithuania|   7.94|
+--------+-------------+-------+
only showing top 15 rows



In [175]:
#Задание 2

df_newcases = df.select('date', 'location', 'new_cases').where((df.date <= '2021-03-31') & (df.date >= '2021-03-25') & (f.length(df.iso_code)==3))
df_newcases = df_newcases.withColumn('row_number',f.row_number().over(w.partitionBy('location').orderBy(df_newcases.new_cases.desc())))
df_newcases = df_newcases.where(df_newcases.row_number == 1).drop('row_number')
df_newcases.orderBy(df_newcases.new_cases.desc()).show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil| 100158.0|
|2021-03-26|United States|  77321.0|
|2021-03-31|        India|  72330.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-26|       Poland|  35145.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-26|        Italy|  24076.0|
|2021-03-25|         Peru|  19206.0|
|2021-03-26|      Ukraine|  18226.0|
+----------+-------------+---------+
only showing top 10 rows



In [176]:
#Задание 3
windowSpec = w.partitionBy(df.iso_code).orderBy(df['date'].asc())
df_delta = df.select('date', 'iso_code', 'new_cases', f.lag(df.new_cases).over(windowSpec)
                     .alias('prev_new_cases')).where((df.date <= '2021-03-31') & (df.date >= '2021-03-25') & (df.iso_code == 'RUS'))
df_delta = df_delta.select('date', 'iso_code', (df_delta.new_cases - df_delta.prev_new_cases).alias('delta'), 'new_cases', 'prev_new_cases')

df_delta.show()

+----------+--------+------+---------+--------------+
|      date|iso_code| delta|new_cases|prev_new_cases|
+----------+--------+------+---------+--------------+
|2021-03-25|     RUS| 359.0|   9128.0|        8769.0|
|2021-03-26|     RUS| -55.0|   9073.0|        9128.0|
|2021-03-27|     RUS|-290.0|   8783.0|        9073.0|
|2021-03-28|     RUS| 196.0|   8979.0|        8783.0|
|2021-03-29|     RUS|-390.0|   8589.0|        8979.0|
|2021-03-30|     RUS|-427.0|   8162.0|        8589.0|
|2021-03-31|     RUS|  -6.0|   8156.0|        8162.0|
+----------+--------+------+---------+--------------+

